# Main ...

 
``config, load, preprocess, train, eval  for  Tick tick bloom``

**Yo DON'T rerun this unless you want to overwrite past models, always fork and do your stuff and 
DON'T forget to change the name**

**``Mission: NNs on expanding avgs of metadata``**

# Load imports and dependencies

In [6]:
import warnings
import sys
import os
import time
import joblib
import random
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, activations, losses, metrics, models, optimizers, callbacks
from category_encoders.target_encoder import TargetEncoder

warnings.filterwarnings('ignore')

In [7]:
# local utilities imports
from tick_tick_bloom_utils import my_keras_rmse, comp_metric, den2sev_map

In [8]:
# wandb stuff for tracking
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_login = user_secrets.get_secret("wandb_bloom_tracker")

import wandb
wandb.login(key=wandb_login)

wandb: Currently logged in as: k_loki. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Config

In [9]:
# dot dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


# Config
config = {}
config = dotdict(config)
config['RANDOM_SEED'] = 18952


config['unique_id'] = int(time.time())
print(f'unique_id: {config.unique_id}')
config['name'] = f'd128_rmse_lndsat8_raw_v2-{config.unique_id}'

config['PROJECT_NAME'] = 'tick-tick-bloom'
# config['DATA_DIR'] = '../data/'
# config['MODEL_DIR'] = '../models/'
config['SAVE_MODEL'] = True


# # Img config
# config['IMG_SIZE'] = (136, 136)
# config['CHANNELS'] = 3

# training configuration
config['train'] =  dotdict({
                        'epochs': 50,
                        'batch_size': 128,
                        'validation_split': 0.2,
                        'shuffle': True,
                        'verbose': 1
                        })

config['desc'] = """ xp3-to see how nns perform only on metadata(use data upto the point of availibility.) with rmse loss""" 

unique_id: 1673364914


In [10]:
# seed everything
random.seed(config.RANDOM_SEED)
np.random.seed(config.RANDOM_SEED)
tf.random.set_seed(config.RANDOM_SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(config.RANDOM_SEED)

# tf.keras.utils.set_random_seed(config.RANDOM_SEED)  # supposedly sets seed for python, numpy, tf

# Load data

In [11]:
INPUT_METADATA_DIR = '/kaggle/input/ticktickbloomdataset'

metadata = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'metadata.csv'))
sub_format = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'submission_format.csv'))
train_labels = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'train_labels.csv'))

In [12]:
# IMG_DIR = '/kaggle/input/pull-landsat-data-v1-500m/landsat8_500m_v1'   # landsat 8 data with raw 
# img_files = os.listdir(IMG_DIR)
# img_file_names = [f.split('.')[0] for f in img_files]

# # get only data for those 1k imgs
# metadata_subset = metadata[metadata['uid'].isin(img_file_names)]
# data = metadata_subset[metadata_subset.split == 'train']
# data = data.merge(train_labels, on='uid')

# test_data = metadata[metadata.split == 'train']

In [13]:
# def get_imgs(uids) :
#     imgs = []
#     for uid in uids:
#         img_arr = np.load(IMG_DIR + f'/{uid}.npy')
#         img_arr = np.transpose(img_arr, (2, 1, 0))
#         # resize img
#         img_arr = cv2.resize(img_arr, config.IMG_SIZE)
#         img_arr = img_arr / 255   # normalizeee bro... other wise it's blowing up the networks...
#         imgs.append(img_arr)
#     return np.array(imgs) 


# def get_np_data(split : float = 0.2, task='train'):
#     """Return np data for training and validation."""
#     if task == 'train':
#         print("Loading train and validation data...")
#         x_train_uids, x_val_uids, y_train, y_val = train_val_split(
#             data['uid'],
#             data.severity,
#             val_size=split,
#             random_state=config.RANDOM_SEED,
#             stratify=data.severity
#         )

#         x_train = get_imgs(x_train_uids)
#         x_val = get_imgs(x_val_uids)

#         return x_train, y_train, x_val, y_val


#     if task == 'test':
#         test_ids = test_data.uids
#         x_test
#         return x_test

In [14]:
# x_train, y_train, x_val, y_val = get_np_data()
# print(y_train.value_counts(normalize=True))
# print(y_val.value_counts(normalize=True))
# print('Done')

In [15]:
#  get data
metadata.date = pd.to_datetime(metadata.date)

region = pd.concat((train_labels, sub_format[['region', 'uid']]), axis=0)

data = pd.merge(metadata, region, on='uid', how='left')
print(data.shape)

(23570, 8)


In [17]:
# seasons
seasons = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1
}


#  most of the samples are collected in the months of June, July, August.

# add date time fts.
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data['week'] = data.date.dt.isocalendar().week
# data['day_of_year'] = data.date.dt.
data['season'] = data.month.map(seasons)

In [18]:
data.sort_values(by='date', inplace=True)

In [19]:
data

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season
4387,evep,44.847993,-93.476318,2013-01-04,train,midwest,1.0,115.0,1,2013,1,1
13644,paev,44.822478,-93.367962,2013-01-04,train,midwest,1.0,1884.0,1,2013,1,1
5566,gdxr,44.877646,-93.557842,2013-01-04,train,midwest,1.0,1416.0,1,2013,1,1
6144,guny,44.878889,-93.490833,2013-01-04,train,midwest,1.0,558.0,1,2013,1,1
5317,fwbt,44.850500,-93.515700,2013-01-04,train,midwest,1.0,476.0,1,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,west,NaN,NaN,12,2021,52,1
17559,thki,36.725400,-121.730000,2021-12-29,test,west,NaN,NaN,12,2021,52,1
17452,teuu,36.772300,-121.788000,2021-12-29,test,west,NaN,NaN,12,2021,52,1
14254,prfi,36.751800,-121.742000,2021-12-29,test,west,NaN,NaN,12,2021,52,1


In [20]:
data['expanding_severity'] = data.severity.expanding().mean()
data['exapnding_severity_int'] = data['expanding_severity'].apply(np.round)
data

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity,exapnding_severity_int
4387,evep,44.847993,-93.476318,2013-01-04,train,midwest,1.0,115.0,1,2013,1,1,1.000000,1.0
13644,paev,44.822478,-93.367962,2013-01-04,train,midwest,1.0,1884.0,1,2013,1,1,1.000000,1.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,midwest,1.0,1416.0,1,2013,1,1,1.000000,1.0
6144,guny,44.878889,-93.490833,2013-01-04,train,midwest,1.0,558.0,1,2013,1,1,1.000000,1.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,midwest,1.0,476.0,1,2013,1,1,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.145955,2.0
17559,thki,36.725400,-121.730000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.145955,2.0
17452,teuu,36.772300,-121.788000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.145955,2.0
14254,prfi,36.751800,-121.742000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.145955,2.0


In [21]:
train_data = data[data.split == 'train']
test_data = data[data.split == 'test']

In [22]:
# expanding avg of severity
mse(train_data.severity, train_data.exapnding_severity_int, squared=False)

1.2085810811762927

In [23]:
grp_by_region = train_data.groupby('region').severity.expanding(1).mean()
grp_by_region = grp_by_region.map(np.round)
print(grp_by_region.isna().sum())
mse(train_data.severity.sort_index(), grp_by_region.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


0.894165010958815

In [24]:
grp_by_region = data.groupby('region').severity.expanding(1).mean()
grp_by_region = grp_by_region.map(np.round)

grp_by_region['west'].fillna(4, inplace=True)
grp_by_region['northeast'].fillna(4, inplace=True)
print(grp_by_region.isna().sum())

mse(train_data.severity.sort_index(), grp_by_region.droplevel(0).loc[train_data.index].sort_index(), squared=False)

0


0.894165010958815

In [26]:
data['expndng_sev_by_reg'] = np.nan

south = data.region == 'south'
midwest = data.region == 'midwest'
northeast = data.region == 'northeast'
west = data.region == 'west'

data.loc[south , 'expndng_sev_by_reg'] = grp_by_region['south']
data.loc[midwest , 'expndng_sev_by_reg'] = grp_by_region['midwest']
data.loc[northeast , 'expndng_sev_by_reg'] = grp_by_region['northeast']
data.loc[west , 'expndng_sev_by_reg'] = grp_by_region['west']

print(data.shape)
data.isna().sum()

(23570, 15)


uid                          0
latitude                     0
longitude                    0
date                         0
split                        0
region                       0
severity                  6510
density                   6510
month                        0
year                         0
week                         0
season                       0
expanding_severity           0
exapnding_severity_int       0
expndng_sev_by_reg           0
dtype: int64

In [124]:
#  Making submission

expanding_sev_by_reg = data[data.split == 'test'][['uid', 'expndng_sev_by_reg']]
expanding_sev_by_reg.expndng_sev_by_reg = expanding_sev_by_reg.expndng_sev_by_reg.astype(int)
expanding_sev_by_reg.sort_values(by='uid', inplace=True)
expanding_sev_by_reg.reset_index(drop=True, inplace=True)
sub_format.severity = expanding_sev_by_reg.expndng_sev_by_reg
sub_format.severity.value_counts()  # expected 0.894165010958815

2    3490
4    2390
1     630
Name: severity, dtype: int64

In [28]:
# sub_format.to_csv('expndng_sev_by_reg_preds.csv', index=False)

In [29]:
grp_by_week = train_data.groupby('week').severity.expanding(1).mean()
grp_by_week = grp_by_week.map(np.round)
print(grp_by_week.isna().sum())
mse(train_data.severity.sort_index(), grp_by_week.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


1.1762848105162917

In [30]:
grp_by_month = train_data.groupby('month').severity.expanding(1).mean()
grp_by_month = grp_by_month.map(np.round)
print(grp_by_month.isna().sum())
mse(train_data.severity.sort_index(), grp_by_month.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


1.2026251083487944

In [31]:
grp_by_season = train_data.groupby('season').severity.expanding(1).mean()
grp_by_season = grp_by_season.map(np.round)
print(grp_by_season.isna().sum())
mse(train_data.severity.sort_index(), grp_by_season.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


1.2082415307865675

In [32]:
grp_by_yr = train_data.groupby('year').severity.expanding(1).mean()
grp_by_yr = grp_by_yr.map(np.round)
grp_by_yr.isna().sum()
mse(train_data.severity.sort_index(), grp_by_yr.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

1.2071009130567592

In [33]:
#  this seems to be much better than expanding avg severity by region.
grp_by_rw = train_data.groupby(['region', 'week']).severity.expanding(1).mean()
grp_by_rw = grp_by_rw.map(np.round)
grp_by_rw.isna().sum()
mse(train_data.severity.sort_index(), grp_by_rw.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0.8209593221806205

In [36]:
52*4

208

In [77]:
data['expndng_sev_by_rw_1'] = np.nan

grp_by_rw = data.groupby(['region', 'week']).severity.expanding(1).mean()
grp_by_rw = grp_by_rw.map(np.round)
print(grp_by_rw.isna().sum())  # 566 missing vaulues 

data['expndng_sev_by_rw_1'] = grp_by_rw.droplevel(0).droplevel(0).sort_index()


# 1 for imputing missing values with expanding_avg_by_reg   # current best option
data.expndng_sev_by_rw_1 = np.where(data.expndng_sev_by_rw_1.isna(), data.expndng_sev_by_reg, data.expndng_sev_by_rw_1)
print(data.isna().sum())

mse(train_data.severity.sort_index(), data.expndng_sev_by_rw_1.loc[data.split=='train'].sort_index(), squared=False)

566
uid                          0
latitude                     0
longitude                    0
date                         0
split                        0
region                       0
severity                  6510
density                   6510
month                        0
year                         0
week                         0
season                       0
expanding_severity           0
exapnding_severity_int       0
expndng_sev_by_reg           0
expndng_sev_by_rw            0
expndng_sev_by_rw_1          0
dtype: int64


0.8209593221806205

In [123]:
#  make submission for expndng_sev_by_rw_1

expndng_sev_by_rw_1 = data[data.split == 'test'][['uid', 'expndng_sev_by_rw_1']]
expndng_sev_by_rw_1.expndng_sev_by_rw_1 = expndng_sev_by_rw_1.expndng_sev_by_rw_1.astype(int)
expndng_sev_by_rw_1.sort_values(by='uid', inplace=True)
expndng_sev_by_rw_1.reset_index(drop=True, inplace=True)
sub_format.severity = expndng_sev_by_rw_1.expndng_sev_by_rw_1
sub_format.severity.value_counts()  # expected 0.8209593221806205 got 0.8971

2    2670
4    1985
1    1162
3     693
Name: severity, dtype: int64

In [126]:
sub_format.to_csv('expndng_sev_by_rw_exreg_preds.csv', index=False)

In [75]:
grp_by_rm = train_data.groupby(['region', 'month']).severity.expanding(1).mean()
grp_by_rm = grp_by_rm.map(np.round)
grp_by_rm.isna().sum()
mse(train_data.severity.sort_index(), grp_by_rm.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0.8519682748690546

In [76]:
grp_by_ry = train_data.groupby(['region', 'year']).severity.expanding(1).mean()
grp_by_ry = grp_by_ry.map(np.round)
grp_by_ry.isna().sum()
mse(train_data.severity.sort_index(), grp_by_ry.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0.8797580398469024

In [77]:
grp_by_rs = train_data.groupby(['region', 'season']).severity.expanding(1).mean()
grp_by_rs = grp_by_rs.map(np.round)
grp_by_rs.isna().sum()
mse(train_data.severity.sort_index(), grp_by_rs.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0.8587184759090294

In [ ]:
grp_by_rs = train_data.groupby(['region', 'season']).severity.expanding(1).mean()
grp_by_rs = grp_by_rs.map(np.round)
grp_by_rs.isna().sum()
mse(train_data.severity.sort_index(), grp_by_rs.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

In [78]:
grp_by_dt = train_data.groupby('date').severity.expanding(1).mean()
grp_by_dt = grp_by_yr.map(np.round)
grp_by_dt.isna().sum()
mse(train_data.severity.sort_index(), grp_by_dt.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

1.2071009130567592

In [82]:
grp_by_rs = train_data.groupby(['region', 'date']).severity.expanding(1).mean()
grp_by_rs = grp_by_rs.map(np.round)
print(grp_by_rs.isna().sum())
mse(train_data.severity.sort_index(), grp_by_rs.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


0.6547240185341351

In [116]:
data['expanding_sev_by_rd'] = np.nan
grp_by_rs = data.groupby(['region', 'date']).severity.expanding(1).mean()
grp_by_rs = grp_by_rs.map(np.round)
print(grp_by_rs.isna().sum()) # 3706

data.expanding_sev_by_rd = np.where(data['expanding_sev_by_rd'].isna(), data.expndng_sev_by_rw_1, data.expanding_sev_by_rd)
data.expanding_sev_by_rd.isna().sum()

mse(train_data.severity.sort_index(), data.expanding_sev_by_rd.loc[data.split=='train'].sort_index(), squared=False)

3706


0.8209593221806205

In [111]:
sum(data.expanding_sev_by_rd == data.expndng_sev_by_rw_1)

23570

In [95]:
data.groupby(['region', 'month']).ngroups # one group is missing!

47

In [104]:
data.where(~data['expanding_sev_by_rd'].isna(), data.expndng_sev_by_rw_1, axis=1).isna().sum()

uid                       23570
latitude                  23570
longitude                 23570
date                      23570
split                     23570
region                    23570
severity                  23570
density                   23570
month                     23570
year                      23570
week                      23570
season                    23570
expanding_severity        23570
exapnding_severity_int    23570
expndng_sev_by_reg        23570
expndng_sev_by_rw         23570
expndng_sev_by_rw_1       23570
expanding_sev_by_rd       23570
dtype: int64

In [ ]:
all_train = data[data.split == 'train']
all_train.shape

In [ ]:
test_data = metadata[metadata.split == 'test']
test_data = test_data.merge(sub_format, on='uid')

test_data['month'] = test_data.date.dt.month
test_data['year'] = test_data.date.dt.year
test_data['season'] = test_data.month.map(seasons)

In [ ]:
train, val = train_test_split(all_train, test_size=0.20, random_state=config.RANDOM_SEED, stratify=all_train.severity)
train.shape, val.shape

In [ ]:
X_train_, y_train_ = train[['region', 'month', 'year', 'season']], train['severity'] 
X_val_, y_val_ = val[['region', 'month', 'year', 'season']], val['severity']

X_train_.shape, y_train_.shape, X_val_.shape, y_val_.shape

In [ ]:
X_test_ = test_data[['region', 'month', 'year', 'season']]
X_test_.shape

# Preprocess

In [ ]:
# # change labels to 0-3(model works this way) instead of 1-4 given range(given severity)
# -1 for to make labels look like sparse encoded labels

y_train = y_train_  -1
y_val = y_val_ - 1


y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)  # guessing alwyas 0 gives 43% acc

In [ ]:
#  target encode the cat fts.

te = TargetEncoder(cols=['region', 'month', 'year', 'season'])
te.fit(X_train_, y_train)
X_train =  te.transform(X_train_)
X_val = te.transform(X_val_)

In [ ]:
X_test = te.transform(X_test_)
X_test

In [ ]:
display(X_train.head())
display(X_val.head())

# Model

In [ ]:
def comp_loss(y_true, y_pred):
    return tf.sqrt(losses.mean_squared_error(y_true, y_pred))

In [ ]:
def get_model():
    print('Loading model...')
#     input_shape = (*config.IMG_SIZE, config.CHANNELS)

    input_imgs = layers.Input(shape=(X_train.shape[1],))
#     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(input_imgs)
    output = layers.Dense(1)(x)     # regressing the severity level !

    model = models.Model(inputs=input_imgs, outputs=output, name=config.name)

    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                    loss=comp_loss,
                    metrics=[
                        comp_loss,
                        metrics.SparseCategoricalAccuracy(name='acc')
                    ])

    return model

In [ ]:
model = get_model()
model.summary()

# Train and eval

In [ ]:
# np and tf metrics are aggreable, but why evaluate and final epoch's loss and metric each, are different?!
# 1.5447352189134953 (np) 1.5447352189134953 (tf) , 1.5175646543502808(evaluate), 


# >> that's coz final epoch also has some batchs which are worst performing and model is still learning and final epoch loss is avg of all those batches.
# whereas evaluete the model's fixed and no leariing believe in evaluate!

In [ ]:
def train(model=get_model(), config=config, x_train=X_train, y_train=y_train):
    """fits given model to x_train and y_train"""
    
    train_config = config['train']
    my_callbacks = []
    
    earlystopping = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    my_callbacks.append(earlystopping)
    
    try:
        wandb_callback = wandb.keras.WandbCallback(
            monitor='val_loss',
            log_weights=True,
            log_gradients=True,
            save_model=False,
            training_data=(x_train, y_train),
            log_batch_frequency=None,
        )

        my_callbacks.append(wandb_callback)
    except:
        print('wandb not tracking')
        
    print(f'Training model... {config.name}')
    history = model.fit(
                x_train, y_train,
                epochs=train_config.epochs,
                batch_size=train_config.batch_size, 
                callbacks=my_callbacks, 
                validation_split=0.2, 
                shuffle=True, 
                verbose=1   
            )

    return model, history

In [ ]:
def eval_model(model, x_val=X_val, y_val=y_val):
    print('Evaluating model....')
    model.evaluate(x_val, y_val, return_dict=True)


In [ ]:
with wandb.init(project=config.PROJECT_NAME, config=config, name=config.name):
    
    # load nwe model or old saved model
    model = get_model()

    # train
    model, history = train(model, config, y_train=y_train)
    # eval
    eval_model(model)
    
    # classification report
    y_pred = model.predict(X_val)
    y_pred_hard = np.round(y_pred)
    error = mse(y_val, y_pred_hard, squared=False)
    print("Comp Metric: ", error)
    cr = classification_report(y_val+1, y_pred_hard+1)     # +1 to account for 0-4 as it should be 1-5 originallly
    print(cr)
    

In [ ]:
# history.history

In [ ]:
model = models.load_model('/kaggle/working/d128_rmse_lndsat8_raw_v1-1673283452.h5', custom_objects={'comp_loss': comp_loss})
preds = model.predict(X_val)
int_preds = np.round(preds)
mse(y_val, int_preds, squared=False)

In [ ]:
int_preds.shape

In [ ]:
pd.Series(int_preds.ravel()).value_counts()  # no 3 and 5

# Save something..

In [ ]:
# save model
if config.SAVE_MODEL:
    model.save(config.name + '.h5')
    print("Model saved as ",config.name + '.h5')

# Make submission

In [ ]:
model.summary()

In [ ]:
test_preds = np.round(model.predict(X_test)).ravel()
test_preds = test_preds + 1
sub_format.severity = test_preds
sub_format.severity = sub_format.severity.astype(int) 
sub_format.severity.value_counts()

In [ ]:
sub_format.to_csv(f'{config.name}_preds.csv', index=False) # expecting @ 0.9777

# So...

- Just realized that should only use data upto point of availibility.
- Expanding avg severity by region @ 0.89 best


# ToDos:

- IS this way of modelling with 1 output neuron better compared to 5 as clf
- Does adding data provide any value??
- **Try to beat expanding avg_severity_by_region with the help of imgs, Other wise no use for img data**
- Is there any other way to extract more info than expanding_avg_severity_by_region only from metadata???